# AI Meme Creator Agent

* An AI agent that generates memes using browser automation and multiple LLM models.
* The agent can create custom memes based on user prompts by navigating to meme creation websites, selecting templates, and generating captions.
* Features include multi-model support (Claude, Deepseek, OpenAI), browser automation, template selection, and automatic meme generation with direct image links.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Dhivya-Bharathy/PraisonAI/blob/main/examples/cookbooks/ai_meme_creator_agent.ipynb)


# Dependencies

In [ ]:
!pip install praisonai streamlit browser-use langchain-openai langchain-anthropic langchain-core

# Setup Key

In [2]:
import os

# Set your API keys
openai_key = "Enter you api key here"
anthropic_key = "Enter you api key here"  # Get from https://console.anthropic.com
deepseek_key = "Enter you api key here"    # Get from https://platform.deepseek.com

# Set environment variables
os.environ["OPENAI_API_KEY"] = openai_key
os.environ["ANTHROPIC_API_KEY"] = anthropic_key
os.environ["DEEPSEEK_API_KEY"] = deepseek_key

# Model selection
model_choice = "OpenAI"  # Options: "OpenAI", "Claude", "Deepseek"

print("✅ API keys configured!")
print(f"✅ Using model: {model_choice}")

✅ API keys configured!
✅ Using model: OpenAI


# Tools

In [3]:
# Custom Meme Template Search Tool
import requests
from typing import Dict, Any, List
import re

class MemeTemplateSearchTool:
    def __init__(self):
        self.base_url = "https://imgflip.com"
        self.popular_templates = [
            "distracted-boyfriend", "two-buttons", "one-does-not-simply",
            "drake-hotline-bling", "change-my-mind", "ancient-aliens",
            "y-u-no", "success-kid", "first-world-problems", "futurama-fry"
        ]

    def search_templates(self, query: str) -> Dict[str, Any]:
        """Search for meme templates based on query"""
        try:
            # Extract key action words from query
            action_words = self._extract_action_words(query.lower())

            # Find relevant templates
            relevant_templates = []
            for template in self.popular_templates:
                if any(word in template for word in action_words):
                    relevant_templates.append(template)

            # If no direct matches, return popular templates
            if not relevant_templates:
                relevant_templates = self.popular_templates[:5]

            return {
                "success": True,
                "query": query,
                "action_words": action_words,
                "suggested_templates": relevant_templates,
                "template_urls": [f"{self.base_url}/{template}" for template in relevant_templates]
            }
        except Exception as e:
            return {"error": f"Error searching templates: {str(e)}"}

    def _extract_action_words(self, text: str) -> List[str]:
        """Extract action words from text"""
        action_words = [
            "laugh", "cry", "smile", "angry", "surprised", "confused",
            "happy", "sad", "excited", "worried", "thinking", "shocked",
            "bully", "fight", "work", "study", "sleep", "eat", "drink",
            "run", "walk", "sit", "stand", "look", "watch", "listen"
        ]

        found_words = []
        for word in action_words:
            if word in text:
                found_words.append(word)

        return found_words

# Custom Caption Generator Tool
import random

class CaptionGeneratorTool:
    def __init__(self):
        self.caption_styles = {
            "setup_punchline": "Top: {setup}\nBottom: {punchline}",
            "comparison": "Top: {option1}\nBottom: {option2}",
            "reaction": "Top: {situation}\nBottom: {reaction}",
            "confession": "Top: {confession}\nBottom: {reality}"
        }

    def generate_captions(self, query: str, template: str) -> Dict[str, Any]:
        """Generate meme captions based on query and template"""
        try:
            # Analyze query for meme type
            meme_type = self._analyze_meme_type(query)

            # Generate captions based on type
            if meme_type == "comparison":
                captions = self._generate_comparison_captions(query)
            elif meme_type == "reaction":
                captions = self._generate_reaction_captions(query)
            elif meme_type == "confession":
                captions = self._generate_confession_captions(query)
            else:
                captions = self._generate_setup_punchline_captions(query)

            return {
                "success": True,
                "query": query,
                "template": template,
                "meme_type": meme_type,
                "captions": captions,
                "suggested_style": self.caption_styles.get(meme_type, self.caption_styles["setup_punchline"])
            }
        except Exception as e:
            return {"error": f"Error generating captions: {str(e)}"}

    def _analyze_meme_type(self, query: str) -> str:
        """Analyze query to determine meme type"""
        query_lower = query.lower()

        if any(word in query_lower for word in ["vs", "versus", "or", "either", "choose"]):
            return "comparison"
        elif any(word in query_lower for word in ["when", "reaction", "face", "look"]):
            return "reaction"
        elif any(word in query_lower for word in ["confess", "secret", "actually", "truth"]):
            return "confession"
        else:
            return "setup_punchline"

    def _generate_setup_punchline_captions(self, query: str) -> List[Dict[str, str]]:
        """Generate setup-punchline style captions"""
        captions = []

        # Extract key elements from query
        words = query.split()
        if len(words) >= 3:
            # Create variations
            captions.append({
                "top": f"When {query}",
                "bottom": "But it actually works"
            })
            captions.append({
                "top": f"Me trying to {query}",
                "bottom": "Reality: Complete failure"
            })
            captions.append({
                "top": f"Everyone: {query}",
                "bottom": "Me: *confused*"
            })

        return captions

    def _generate_comparison_captions(self, query: str) -> List[Dict[str, str]]:
        """Generate comparison style captions"""
        captions = []

        captions.append({
            "top": "What I think will happen",
            "bottom": "What actually happens"
        })
        captions.append({
            "top": "The plan",
            "bottom": "The execution"
        })
        captions.append({
            "top": "Before",
            "bottom": "After"
        })

        return captions

    def _generate_reaction_captions(self, query: str) -> List[Dict[str, str]]:
        """Generate reaction style captions"""
        captions = []

        captions.append({
            "top": f"When {query}",
            "bottom": "My face: 😱"
        })
        captions.append({
            "top": f"Me: {query}",
            "bottom": "Also me: *surprised pikachu*"
        })

        return captions

    def _generate_confession_captions(self, query: str) -> List[Dict[str, str]]:
        """Generate confession style captions"""
        captions = []

        captions.append({
            "top": f"I confess: {query}",
            "bottom": "But I'm not sorry"
        })
        captions.append({
            "top": f"Everyone thinks {query}",
            "bottom": "But actually..."
        })

        return captions

# Custom Meme Validation Tool
class MemeValidationTool:
    def __init__(self):
        self.validation_criteria = {
            "humor": ["funny", "amusing", "entertaining"],
            "relevance": ["related", "connected", "appropriate"],
            "clarity": ["clear", "understandable", "readable"],
            "creativity": ["original", "creative", "unique"]
        }

    def validate_meme(self, query: str, captions: List[Dict[str, str]], template: str) -> Dict[str, Any]:
        """Validate meme quality and suggest improvements"""
        try:
            scores = {}
            suggestions = []

            # Check humor
            humor_score = self._check_humor(query, captions)
            scores["humor"] = humor_score
            if humor_score < 0.7:
                suggestions.append("Consider making the punchline more unexpected or relatable")

            # Check relevance
            relevance_score = self._check_relevance(query, captions)
            scores["relevance"] = relevance_score
            if relevance_score < 0.8:
                suggestions.append("Ensure captions directly relate to the query topic")

            # Check clarity
            clarity_score = self._check_clarity(captions)
            scores["clarity"] = clarity_score
            if clarity_score < 0.9:
                suggestions.append("Make sure text is short and easy to read")

            # Check creativity
            creativity_score = self._check_creativity(query, captions)
            scores["creativity"] = creativity_score
            if creativity_score < 0.6:
                suggestions.append("Try a more unique angle or unexpected twist")

            # Overall score
            overall_score = sum(scores.values()) / len(scores)

            return {
                "success": True,
                "scores": scores,
                "overall_score": overall_score,
                "suggestions": suggestions,
                "quality": "Excellent" if overall_score >= 0.8 else "Good" if overall_score >= 0.6 else "Needs Improvement"
            }
        except Exception as e:
            return {"error": f"Error validating meme: {str(e)}"}

    def _check_humor(self, query: str, captions: List[Dict[str, str]]) -> float:
        """Check humor quality"""
        # Simple heuristic - check for contrast, exaggeration, or relatable elements
        score = 0.5  # Base score

        for caption in captions:
            text = f"{caption.get('top', '')} {caption.get('bottom', '')}".lower()

            # Check for humor indicators
            if any(word in text for word in ["but", "however", "actually", "reality"]):
                score += 0.2  # Contrast
            if any(word in text for word in ["everyone", "nobody", "always", "never"]):
                score += 0.1  # Exaggeration
            if any(word in text for word in ["me", "my", "i", "we"]):
                score += 0.1  # Relatable

        return min(1.0, score)

    def _check_relevance(self, query: str, captions: List[Dict[str, str]]) -> float:
        """Check relevance to query"""
        query_words = set(query.lower().split())
        score = 0.5  # Base score

        for caption in captions:
            caption_text = f"{caption.get('top', '')} {caption.get('bottom', '')}".lower()
            caption_words = set(caption_text.split())

            # Check word overlap
            overlap = len(query_words.intersection(caption_words))
            if overlap > 0:
                score += 0.3
            if overlap > 2:
                score += 0.2

        return min(1.0, score)

    def _check_clarity(self, captions: List[Dict[str, str]]) -> float:
        """Check text clarity and readability"""
        score = 1.0  # Start with perfect score

        for caption in captions:
            top_text = caption.get('top', '')
            bottom_text = caption.get('bottom', '')

            # Penalize long text
            if len(top_text) > 50:
                score -= 0.1
            if len(bottom_text) > 50:
                score -= 0.1

            # Penalize complex words
            complex_words = [word for word in top_text.split() if len(word) > 10]
            if len(complex_words) > 0:
                score -= 0.1

        return max(0.0, score)

    def _check_creativity(self, query: str, captions: List[Dict[str, str]]) -> float:
        """Check creativity and originality"""
        score = 0.5  # Base score

        # Check for common meme phrases (penalize)
        common_phrases = [
            "surprised pikachu", "change my mind", "one does not simply",
            "ancient aliens", "distracted boyfriend", "drake"
        ]

        for caption in captions:
            caption_text = f"{caption.get('top', '')} {caption.get('bottom', '')}".lower()

            # Reward for avoiding common phrases
            if not any(phrase in caption_text for phrase in common_phrases):
                score += 0.3

            # Reward for unique word combinations
            unique_words = len(set(caption_text.split()))
            if unique_words > 8:
                score += 0.2

        return min(1.0, score)

# YAML Prompt

In [4]:
# YAML Prompt
yaml_prompt = """
name: "AI Meme Creator Agent"
description: "Expert meme generator with browser automation and multi-model support"
instructions:
  - "You are an expert meme creator that generates funny and relevant memes based on user prompts"
  - "Use browser automation to navigate meme creation websites and select appropriate templates"
  - "Generate creative captions that match the meme template and user's request"
  - "Ensure memes are humorous, relevant, and visually appealing"
  - "Provide multiple caption options for different meme styles"
  - "Validate meme quality and suggest improvements when needed"
  - "Use appropriate meme templates that fit the context and emotion"
  - "Create captions that are short, readable, and impactful"
  - "Consider current trends and popular meme formats"

tools:
  - name: "MemeTemplateSearchTool"
    description: "Searches for appropriate meme templates based on user query and action words"
  - name: "CaptionGeneratorTool"
    description: "Generates creative captions for memes based on query, template, and meme type"
  - name: "MemeValidationTool"
    description: "Validates meme quality and provides suggestions for improvement"

output_format:
  - "Provide the generated meme image URL"
  - "Include caption suggestions and template recommendations"
  - "Offer quality assessment and improvement tips"
  - "Explain the meme concept and why it works"
  - "Provide alternative caption options if available"

temperature: 0.7
max_tokens: 3000
model: "gpt-5-nano"
"""

print("✅ YAML Prompt configured!")

✅ YAML Prompt configured!


# Main

In [6]:
import asyncio
import streamlit as st
from browser_use import Agent, SystemPrompt
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage
import re
import os
from typing import Dict, Any, List

# Initialize tools
template_tool = MemeTemplateSearchTool()
caption_tool = CaptionGeneratorTool()
validation_tool = MemeValidationTool()

print("🥸 AI Meme Creator Agent")
print("Generate hilarious memes using AI and browser automation!")

# Model configuration
print("\n⚙️ Model Configuration")
model_choice = "OpenAI"  # Options: "OpenAI", "Claude", "Deepseek"
print(f"✅ Using model: {model_choice}")

# Meme generation function
async def generate_meme(query: str, model_choice: str, api_key: str) -> str:
    """Generate meme using browser automation"""
    try:
        # Initialize the appropriate LLM based on user selection
        if model_choice == "Claude":
            llm = ChatAnthropic(
                model="claude-3-5-sonnet-20241022",
                api_key=api_key
            )
        elif model_choice == "Deepseek":
            llm = ChatOpenAI(
                base_url='https://api.deepseek.com/v1',
                model='deepseek-chat',
                api_key=api_key,
                temperature=0.3
            )
        else:  # OpenAI
            llm = ChatOpenAI(
                model="gpt-4o",
                api_key=api_key,
                temperature=0.0
            )

        task_description = (
            "You are a meme generator expert. You are given a query and you need to generate a meme for it.\n"
            "1. Go to https://imgflip.com/memetemplates \n"
            "2. Click on the Search bar in the middle and search for ONLY ONE MAIN ACTION VERB (like 'bully', 'laugh', 'cry') in this query: '{0}'\n"
            "3. Choose any meme template that metaphorically fits the meme topic: '{0}'\n"
            "   by clicking on the 'Add Caption' button below it\n"
            "4. Write a Top Text (setup/context) and Bottom Text (punchline/outcome) related to '{0}'.\n"
            "5. Check the preview making sure it is funny and a meaningful meme. Adjust text directly if needed. \n"
            "6. Look at the meme and text on it, if it doesnt make sense, PLEASE retry by filling the text boxes with different text. \n"
            "7. Click on the Generate meme button to generate the meme\n"
            "8. Copy the image link and give it as the output\n"
        ).format(query)

        agent = Agent(
            task=task_description,
            llm=llm,
            max_actions_per_step=5,
            max_failures=25,
            use_vision=(model_choice != "Deepseek")
        )

        history = await agent.run()

        # Extract final result from agent history
        final_result = history.final_result()

        # Use regex to find the meme URL in the result
        url_match = re.search(r'https://imgflip\.com/i/(\w+)', final_result)
        if url_match:
            meme_id = url_match.group(1)
            return f"https://i.imgflip.com/{meme_id}.jpg"
        return None
    except Exception as e:
        print(f"Error in meme generation: {str(e)}")
        return None

# Interactive meme creation
print("\n🎨 Meme Creation Interface")
print("Enter your meme idea:")

query = input("Meme Idea: ")

if query.strip():
    print(f"\n🚀 Generating meme for: '{query}'")

    # Step 1: Search for templates
    print("\n🔍 Step 1: Searching for meme templates...")
    template_result = template_tool.search_templates(query)

    if "error" not in template_result:
        print(f"✅ Found {len(template_result['suggested_templates'])} relevant templates")
        print(f"Action words: {', '.join(template_result['action_words'])}")

        # Step 2: Generate captions
        print("\n✍️ Step 2: Generating captions...")
        selected_template = template_result['suggested_templates'][0]
        caption_result = caption_tool.generate_captions(query, selected_template)

        if "error" not in caption_result:
            print(f"✅ Generated {len(caption_result['captions'])} caption options")
            print(f"Meme type: {caption_result['meme_type']}")

            # Display caption options
            print("\n📝 Caption Options:")
            for i, caption in enumerate(caption_result['captions'], 1):
                print(f"Option {i}:")
                print(f"  Top: {caption.get('top', 'N/A')}")
                print(f"  Bottom: {caption.get('bottom', 'N/A')}")

            # Step 3: Validate meme quality
            print("\n�� Step 3: Validating meme quality...")
            validation_result = validation_tool.validate_meme(
                query, caption_result['captions'], selected_template
            )

            if "error" not in validation_result:
                print(f"✅ Quality Assessment: {validation_result['quality']}")
                print(f"Overall Score: {validation_result['overall_score']:.2f}/1.0")

                if validation_result['suggestions']:
                    print("💡 Suggestions for improvement:")
                    for suggestion in validation_result['suggestions']:
                        print(f"  - {suggestion}")

            # Step 4: Generate actual meme (if API key is available)
            print("\n�� Step 4: Generating meme with browser automation...")

            # Check if we have the required API key
            api_key = os.getenv("OPENAI_API_KEY")
            if api_key and api_key != "your_openai_key_here":
                try:
                    print("🔄 Starting browser automation...")
                    meme_url = await generate_meme(query, model_choice, api_key)

                    if meme_url:
                        print("✅ Meme Generated Successfully!")
                        print(f"📸 Meme URL: {meme_url}")
                        print(f"�� Direct Link: {meme_url}")

                        # Display meme (in Colab, we can use HTML)
                        from IPython.display import Image, display
                        display(Image(url=meme_url))
                    else:
                        print("❌ Failed to generate meme. This might be due to:")
                        print("  - Browser automation issues")
                        print("  - Website changes")
                        print("  - Network connectivity")
                        print("  - API rate limits")

                        # Provide manual instructions
                        print("\n💡 Manual Meme Creation Instructions:")
                        print("1. Go to https://imgflip.com/memetemplates")
                        print("2. Search for a template related to your query")
                        print("3. Use one of the generated captions:")
                        for i, caption in enumerate(caption_result['captions'], 1):
                            print(f"   Option {i}: Top='{caption.get('top', '')}', Bottom='{caption.get('bottom', '')}'")

                except Exception as e:
                    print(f"❌ Error during meme generation: {str(e)}")
                    print("💡 Try using the manual instructions above")
            else:
                print("⚠️ API key not configured for browser automation")
                print("💡 Using generated captions for manual meme creation:")
                for i, caption in enumerate(caption_result['captions'], 1):
                    print(f"   Option {i}: Top='{caption.get('top', '')}', Bottom='{caption.get('bottom', '')}'")
        else:
            print(f"❌ Error generating captions: {caption_result['error']}")
    else:
        print(f"❌ Error searching templates: {template_result['error']}")

else:
    print("❌ No meme idea provided.")

print("\n🧪 Sample Meme Ideas for Testing")
sample_ideas = [
    "When you finally understand the code after 3 hours of debugging",
    "Me trying to explain AI to my grandparents",
    "The difference between what I plan to do vs what I actually do",
    "When someone asks if I'm free this weekend",
    "My brain at 3 AM thinking about that embarrassing thing from 5 years ago"
]

print("Try these sample ideas:")
for i, idea in enumerate(sample_ideas, 1):
    print(f"{i}. {idea}")

print("\n" + "="*50)
print("�� Powered by AI Meme Creator Agent | Built with PraisonAI")

🥸 AI Meme Creator Agent
Generate hilarious memes using AI and browser automation!

⚙️ Model Configuration
✅ Using model: OpenAI

🎨 Meme Creation Interface
Enter your meme idea:
Meme Idea: When you finally understand the code after 3 hours of debugging

🚀 Generating meme for: 'When you finally understand the code after 3 hours of debugging'

🔍 Step 1: Searching for meme templates...
✅ Found 5 relevant templates
Action words: stand

✍️ Step 2: Generating captions...
✅ Generated 2 caption options
Meme type: reaction

📝 Caption Options:
Option 1:
  Top: When When you finally understand the code after 3 hours of debugging
  Bottom: My face: 😱
Option 2:
  Top: Me: When you finally understand the code after 3 hours of debugging
  Bottom: Also me: *surprised pikachu*

�� Step 3: Validating meme quality...
✅ Quality Assessment: Excellent
Overall Score: 0.88/1.0
💡 Suggestions for improvement:
  - Make sure text is short and easy to read

�� Step 4: Generating meme with browser automation...
🔄 St